In [1]:
import http.client
import os
import re
import sys
import requests

from bs4 import BeautifulSoup
from getpass import getpass
from time import sleep, time as now
from urllib.parse import urljoin
from urllib.request import urlopen

In [2]:
# Set this to 1 for debugging HTTP server communication
http.client.HTTPConnection.debuglevel = 0

In [3]:
bamboo_base_url = "https://bamboo.ec2.local"
local_dir = os.path.expanduser('~/traceability')
security_certificate = os.path.expanduser("~/Americas-Root.pem")
# ec2_username = input('user: ')
# ec2_password = getpass()
ec2_username = "automatedservices"
ec2_password = "EC2RO-ResMed123+"

In [4]:
def get_dir_name_from_url(url):
    m = re.match(r'.*/(build-\d+/Acceptance-Test-Results-\d+)/?', url)
    return m.group(1) if m else None

In [5]:
def download_report(results_url):
    target_dir = os.path.join(local_dir, get_dir_name_from_url(results_url))
    os.makedirs(target_dir, exist_ok=True)
    url = f'{results_url}/cucumberReports/report.json'
    print(f"Downloading {url} ...", end='')

    response = requests.get(url,
                            auth=(ec2_username, ec2_password),
                            verify=security_certificate)
    with open(os.path.join(target_dir, 'report.json'), mode='wb') as f:
        f.write(response.content)
    print('done!')

In [6]:
def extract_links_from_contents(contents):
    soup = BeautifulSoup(markup=contents, features="html")
    links = soup.find_all("a")
    return [
        link['href'] for link in links
        if re.match(r'.*Acceptance-Test-Results-\d+', str(link.contents))
    ]

In [7]:
def download_reports(build_number):
    # https://bamboo.ec2.local/artifact/MON-ATEST/shared/build-2059/
    build_path = f"artifact/MON-ATEST/shared/build-{build_number}/"
    bamboo_build_url = urljoin(bamboo_base_url, build_path)
    response = requests.get(bamboo_build_url,
                            auth=(ec2_username, ec2_password),
                            verify=security_certificate)
    refs = extract_links_from_contents(response.content)
    for ref in refs:
        download_report(urljoin(bamboo_base_url, ref))

In [9]:
download_reports(input('Build number:'))

Build number:2059


In [27]:
def get_local_reports(build_number):
    """Returns the file path of all the reports.json files 
    that belong to a build"""
    root = os.path.join(local_dir, f'build-{build_number}')
    result = []
    for dir_path, dir_names, file_names in os.walk(root):
        if file_names and 'report.json' in file_names:
            result.append(os.path.join(dir_path, 'report.json'))
    result.sort()
    return result

In [28]:
get_local_reports(input('build number:'))

build number:2058


['/Users/erikc/traceability/build-2058/Acceptance-Test-Results-01/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-02/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-03/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-04/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-05/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-06/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-07/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-08/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-09/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-10/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-11/report.json',
 '/Users/erikc/traceability/build-2058/Acceptance-Test-Results-12/report.json',
 '/Users/erikc/traceability/build-2058/A

In [36]:
transform_url = 'https://taas-traceability-service.apps.us6.paas.easycareonline.com/taas/traceability/report/transform'
file_name = get_local_reports(2059)[0]
response = requests.post(transform_url, files=dict(file=file_name), auth=(ec2_username, ec2_password),
                            verify=security_certificate)
response

<Response [406]>

In [37]:
response.content

b'{"timestamp":"2019-11-21 10:53:12","description":"Error occured during processing report","error":null}'